In [ ]:
# Get the latest kgtk-browser from GitHub
!rm -rf kgtk-browser && git clone https://github.com/usc-isi-i2/kgtk-browser

# Install the requirements (i.e. kgtk)
!pip install -r kgtk-browser/requirements.txt

In [15]:
from kgtk.functions import kgtk, kypher
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
import json
import os, sqlite3
from google.colab.output import eval_js

ModuleNotFoundError: No module named 'google'

In [4]:
extra_files_config = {
    "claims": "claims.tsv.gz", 
    "labels": "labels.tsv.gz",
    "aliases": "aliases.tsv.gz", 
    "descriptions": "descriptions.tsv.gz",
    "qualifiers": "qualifiers.tsv.gz",
    "metadata": "metadata.tsv.gz"
}

open('/tmp/extra_files.json', 'w').write(json.dumps(extra_files_config))

188

In [ ]:
files = ['claims', 'labels', 'aliases', 'descriptions', 'qualifiers', 'metadata']

input_files_url="https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/kgtk-browser"

input_path = None
output_path = '/tmp/projects'
project_name = 'kgtk-browser'


# Minimal KGTK configuration for this example
ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  json_config_file = '/tmp/extra_files.json')



In [7]:
# Review all of the environment variables used:

ck.print_env_variables()

USE_CASES_DIR: /Users/amandeep/Github/kgtk-notebooks/use-cases
OUT: /tmp/projects/kgtk-browser
TEMP: /tmp/projects/kgtk-browser/temp.kgtk-browser
STORE: /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
GRAPH: /Users/amandeep/isi-kgtk-tutorial/input
kgtk: kgtk
kypher: kgtk query --graph-cache /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/amandeep/Github/kgtk-notebooks/examples
claims: /Users/amandeep/isi-kgtk-tutorial/input/claims.tsv.gz
labels: /Users/amandeep/isi-kgtk-tutorial/input/labels.tsv.gz
aliases: /Users/amandeep/isi-kgtk-tutorial/input/aliases.tsv.gz
descriptions: /Users/amandeep/isi-kgtk-tutorial/input/descriptions.tsv.gz
qualifiers: /Users/amandeep/isi-kgtk-tutorial/input/qualifiers.tsv.gz
metadata: /Users/amandeep/isi-kgtk-tutorial/input/metadata.tsv.gz


In [8]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db -i "/Users/amandeep/isi-kgtk-tutorial/input/claims.tsv.gz" --as claims  -i "/Users/amandeep/isi-kgtk-tutorial/input/labels.tsv.gz" --as labels  -i "/Users/amandeep/isi-kgtk-tutorial/input/aliases.tsv.gz" --as aliases  -i "/Users/amandeep/isi-kgtk-tutorial/input/descriptions.tsv.gz" --as descriptions  -i "/Users/amandeep/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/Users/amandeep/isi-kgtk-tutorial/input/metadata.tsv.gz" --as metadata  --limit 3
node1	label	node2	id
P10	P31	Q18610173	P10-P31-Q18610173-85ef4d24-0
P10	Pdirected_pagerank	2.2996980087906936e-06	P10-Pdirected_pagerank-2
P10	Pout_degree	1	P10-Pout_degree-6b86b2


In [11]:
conn = sqlite3.connect(os.environ['STORE'])

In [ ]:
#  Build the "node1;upper" column in the label table
conn.execute('ALTER TABLE graph_2 ADD COLUMN "node1;upper" text')
conn.commit()

conn.execute('UPDATE graph_2 SET "node1;upper" = upper(node1)')
conn.commit()

# Index the node1;upper column. It supports case-insensitive searches on item labels
conn.execute('CREATE INDEX "graph_2_node1upper_idx" on graph_2 ("node1;upper")')
conn.commit()

conn.execute('ANALYZE "graph_2_node1upper_idx"')
conn.commit()

# Build the "node2;upper" column in the label table
conn.execute('ALTER TABLE graph_2 ADD COLUMN "node2;upper" text')
conn.commit()

conn.execute('UPDATE graph_2 SET "node2;upper" = upper(node2)')
conn.commit()

# Index the node2;upper column. It supports case-insensitive searches on item labels
conn.execute('CREATE INDEX "graph_2_node2upper_idx" on graph_2 ("node2;upper")')
conn.commit()

conn.execute('ANALYZE "graph_2_node2upper_idx"')
conn.commit()


In [14]:
# Verify that the graph cache has loaded as expected
kgtk("""
  query --show-cache \
    --graph-cache ${STORE}
""")

Graph Cache:
DB file: /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
  size:  171.91 MB   	free:  0 Bytes   	modified:  2021-10-23 14:08:00

KGTK File Information:
aliases:
  size:  1.28 MB   	modified:  2021-10-23 14:06:10   	graph:  graph_3
claims:
  size:  31.25 MB   	modified:  2021-10-23 14:06:06   	graph:  graph_1
descriptions:
  size:  1.28 MB   	modified:  2021-10-23 14:06:11   	graph:  graph_4
labels:
  size:  1.02 MB   	modified:  2021-10-23 14:06:08   	graph:  graph_2
metadata:
  size:  20.69 KB   	modified:  2021-10-23 14:06:14   	graph:  graph_6
qualifiers:
  size:  5.73 MB   	modified:  2021-10-14 16:04:41   	graph:  graph_5

Graph Table Information:
graph_1:
  size:  118.56 MB   	created:  2021-10-23 14:07:06
  header:  ['node1', 'label', 'node2', 'id']
graph_2:
  size:  4.09 MB   	created:  2021-10-23 14:07:07
  header:  ['node1', 'label', 'node2', 'id']
graph_3:
  size:  5.27 MB   	created:  2021-10-23 14:07:08
  header:  ['node1', 'label', 'node2', '

In [ ]:
# Get a unique link pointing to this google colab environment

print(eval_js("google.colab.kernel.proxyPort(8000)") + "browser")

In [ ]:
# Set the environment variables for flask
os.environ['FLASK_ENV'] = 'production'
os.environ['FLASK_APP'] = 'kgtk-browser/kgtk_browser_app.py'
os.environ['KGTK_BROWSER_CONFIG'] = 'kgtk_browser_config.py'

# Run the kgtk-browser app
!flask run --port 8000